# Advanced Querying Mongo

Importing libraries and setting up connection

In [23]:
from pymongo import MongoClient

In [31]:
client = MongoClient("localhost:27017")

In [32]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [33]:
db = client.get_database('Ironhack')

In [34]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack')

In [35]:
collection = db.get_collection("companies")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [55]:
# Your Code
filtro = {"name": {"$regex": ".*Babelgum.*"}}
proj = {"_id":0, "name":1}
list(collection.find(filtro, proj))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [49]:
# Your Code
proj = {"_id":0, "name":1, "number_of_employees":1}
list(collection.find({"number_of_employees": {"$gt":5000}}, proj).sort("number_of_employees", 1).limit(20))

[{'name': 'Nintendo', 'number_of_employees': 5080},
 {'name': 'Hexaware Technologies', 'number_of_employees': 5200},
 {'name': 'Facebook', 'number_of_employees': 5299},
 {'name': 'OpenText', 'number_of_employees': 5300},
 {'name': 'LSI', 'number_of_employees': 5400},
 {'name': 'CPM Braxis', 'number_of_employees': 5400},
 {'name': 'Microchip Technologies', 'number_of_employees': 5500},
 {'name': 'Mediaset', 'number_of_employees': 5729},
 {'name': 'Mindray Medical International', 'number_of_employees': 5763},
 {'name': 'Baidu', 'number_of_employees': 6000},
 {'name': 'Dentsu', 'number_of_employees': 6000},
 {'name': 'Tata Communications', 'number_of_employees': 6000},
 {'name': 'Atmel', 'number_of_employees': 6000},
 {'name': 'Acxiom', 'number_of_employees': 6200},
 {'name': 'Acxiom', 'number_of_employees': 6200},
 {'name': 'SRA International', 'number_of_employees': 6400},
 {'name': "Moody's", 'number_of_employees': 6800},
 {'name': 'ManTech', 'number_of_employees': 7000},
 {'name': 'Ad

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [58]:
# Your Code
proj = {"_id":0, "name":1, "founded_year":1}
list(collection.find({"founded_year": {"$in" :[2000, 2001, 2002, 2003, 2004, 2005]}}, proj))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'SmugMug', 'founded_year': 2002},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'Skype', 'founded_year': 2003},
 {'name': 'YouTube', 'founded_year': 2005},
 {'name': 'Pando Networks', 'founded_year': 2004},
 {'name': 'Ikan', 'foun

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [63]:
# Your Code
proj = {"_id": 0, "name": 1, "ipo":1}
filtro = {"$and": [{"ipo.valuation_amount": {"$gt": 100000000}},{"founded_year": {"$lt":2010}}]}

list(collection.find(filtro, proj))

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}},
 {'name': 'Brightcove',
  'ipo': {

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [66]:
# Your Code
proj = {"_id": 0, "name": 1, "number_of_employees":1}
filtro = {"$and": [{"number_of_employees": {"$lt": 1000}},{"founded_year": {"$lt":2005}}]}

list(collection.find(filtro, proj).sort("number_of_employees",-1).limit(10))

[{'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931},
 {'name': 'Forrester Research', 'number_of_employees': 903},
 {'name': 'Webmetrics', 'number_of_employees': 900},
 {'name': 'SonicWALL', 'number_of_employees': 900},
 {'name': 'Cornerstone OnDemand', 'number_of_employees': 881},
 {'name': 'Buongiorno', 'number_of_employees': 800},
 {'name': 'Cvent', 'number_of_employees': 800},
 {'name': 'ZoomInfo', 'number_of_employees': 800}]

### 6. All the companies that don't include the `partners` field.

In [74]:
# Your Code
proj = {"_id": 0, "name": 1}
filtro = {"partners": {"$exists": False}}

list(collection.find(filtro, proj))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [80]:
# Your Code
#collection.distinct("category_code")
proj = {"_id": 0, "name": 1}
filtro = {"category_code": None}
list(collection.find(filtro, proj))

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'},
 {'name': 'Drigg'},
 {'name': 'SpaceTime'},
 {'name': 'Touch Clarity'},
 {'name': 'MMDAYS'},
 {'name': 'Inside Group'},
 {'name': 'Repeater Store'},
 {'name': 'Tapesh'},
 {'name': 'iPersians'},
 {'name': 'NewPersia'},
 {'name': 'Pyra Labs'},
 {'name': 'Feedmap'},
 {'name': 'NuvoMedia'},
 {'name': 'Intwine'},
 {'name': 'The Weinstein Company'},
 {'name': 'ExecuNet'},
 {'name': 'A1Vacations'},
 {'name': 'TripHomes'},
 {'name': 'Picturehouse'},
 {'name': 'Covalent Technologies'},
 {'name': 'Epic Trip'},
 {'name': 'Telepost'},
 {'name': 'Fidelity'},
 {'name': 'Quirk'},
 {'name': 'Skyrove'},
 {'name': 'Leechblock'},
 {'name': 'My Success Gateway'},
 {'name': 'RealNames'},
 {'name': 'The Easynet Group'},
 {'name': 'NetNames'},
 {'name': 'Sniperoo'},
 {'name': 'JD Project'},
 {'name': 'ABC News'},
 {'name': 'Teoma Technologies'},
 {'name': 'cometro'},
 {'name': 'U-Cast'},
 {'na

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [96]:
# Your Code
proj = {"_id": 0, "name": 1, "number_of_employees":1}
filtro = {"$and": [{"number_of_employees": {"$gte": 100}}, {"number_of_employees": {"$lte": 1000}}]}
list(collection.find(filtro, proj))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120},
 {'name': 'RockYou', 'number_of_employees': 106},
 {'name': 'Meebo', 'number_of_employees': 200},
 {'name': 'Eventbrite', 'number_of_employees': 200},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'Conduit', 'number_of_employees': 215},
 {'name': 'Redfin', 'number_of_employees': 100},
 {'name': 'oDesk', 'number_of_employees': 120},
 {'name': 'Simply Hired', 'number_of_employees': 100},
 {'name': 'PhotoBox', 'number_of_employees': 600},
 {'name': 'Spreadshirt', 'number_of_employees'

### 9. Order all the companies by their IPO price in a descending order.

In [102]:
# Your Code
#collection.distinct("ipo.valuation_amount")
proj = {"_id": 0, "name": 1, "ipo.valuation_amount": 1}
filtro = {"ipo.valuation_amount": {"$ne": None}}
list(collection.find(filtro, proj).sort("ipo.valuation_amount", -1))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000}},
 {'name': 'Baidu', 'ipo': {'valuation_amount': 4000000000}},
 {'name': 'TripAdvisor', 'ipo': {'valuation_amount': 3273770000}},
 {'name': 'HomeAway', 'ipo': {'valuation_amount': 3000000000}},
 {'name': 'Zillow', 'ipo': {'valuation_amount': 2550000000}},
 {'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'name': 'Yelp', 'ipo': {'valu

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [106]:
# Your Code
proj = {"_id": 0, "name": 1, "number_of_employees": 1}
filtro = {"number_of_employees": {"$gt": 1}}
list(collection.find(filtro, proj).sort("number_of_employees", -1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [109]:
# Your Code
proj = {"_id": 0, "name": 1, "founded_month": 1}
list(collection.find({"founded_month": {"$in" : [7, 8, 9, 10, 11, 12]}}, proj).limit(1000))

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'eBay', 'founded_month': 9},
 {'name': 'Joost', 'founded_month': 10},
 {'name': 'Plaxo', 'founded_month': 11},
 {'name': 'Powerset', 'founded_month': 10},
 {'name': 'Kyte', 'founded_month': 12},
 {'name': 'Thoof', 'founded_month': 12},
 {'name': 'Jingle Networks', 'founded_month': 9},
 {'name': 'LifeLock', 'founded_month': 8},
 {'name': 'Wesabe', 'founded_month': 12},
 {'name': 'SmugMug', 'founded_month': 11},
 {'name': 'Google', 'founded_month': 9},
 {'name': 'Skype', 'founded_month': 8},
 {'name': 'Pando Networks', 'founded_month': 7},
 {'name': 'Ikan', 'founded_month': 12},
 {'name': 'delicious', 'founded_month': 9},
 {'name': 'AllPeers', 'founded_month': 8},
 {'name': 'Wize', 'founded_month': 8},
 {'name': 'SellABand', 'founded_month': 8},
 {'name': 'iContact', 'founded_month': 7},
 {'name': 'MeeVee', 'foun

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [134]:
# Your Code
proj = {"_id": 0, "name": 1}
filtro = {"$and": [{"founded_year": {"$lt": 2000}},{"acquisition.price_amount": {"$gt":10000}}]}
list(collection.find(filtro, proj))

[{'name': 'Postini'},
 {'name': 'SideStep'},
 {'name': 'Recipezaar'},
 {'name': 'Cyworld'},
 {'name': 'PayPal'},
 {'name': 'Snapfish'},
 {'name': 'Neopets'},
 {'name': 'Sun Microsystems'},
 {'name': 'Zappos'},
 {'name': 'Alibaba'},
 {'name': 'Sabre'},
 {'name': 'Lastminute'},
 {'name': 'Sportsline'},
 {'name': 'Digitas'},
 {'name': 'Webshots'},
 {'name': 'MIVA'},
 {'name': 'iVillage'},
 {'name': 'PriceGrabber'},
 {'name': 'LinkShare'},
 {'name': 'Kaboose'},
 {'name': 'Fastclick'},
 {'name': 'Eloqua'},
 {'name': 'Omniture'},
 {'name': 'Interwoven'},
 {'name': 'HowStuffWorks'},
 {'name': 'RealAge'},
 {'name': 'Match'},
 {'name': 'Xdrive'},
 {'name': 'Alexa'},
 {'name': 'Offermatica'},
 {'name': 'ioko'},
 {'name': 'MyNewPlace'},
 {'name': 'DoubleClick'},
 {'name': 'Danger'},
 {'name': 'Direct Hit'},
 {'name': 'BEA Systems'},
 {'name': 'Napster'},
 {'name': 'SafeNet'},
 {'name': 'LowerMyBills'},
 {'name': 'The FeedRoom'},
 {'name': 'Various'},
 {'name': 'ServerBeach'},
 {'name': 'aQuantive

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [121]:
# Your Code
proj = {"_id": 0, "name": 1, "acquisition": 1}
filtro = {"$and": [{"acquisition.acquired_year": {"$gt":2010}}, {"acquisition.price_amount":{"$ne": None}}]}
list(collection.find(filtro, proj).sort("acquisition.price_amount", -1))

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name': 'LS

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [125]:
# Your Code
#collection.distinct("founded_year")
proj = {"_id": 0, "name": 1, "founded_year": 1}
filtro = {"founded_year": {"$ne": None}}
list(collection.find(filtro, proj).sort("founded_year", -1))

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'Tongxue', 'founded_year': 2013},
 {'name': 'Shopseen', 'founded_year': 2013},
 {'name': 'VistaGen Therapeutics', 'founded_year': 2013},
 {'name': 'PeekYou', 'founded_year': 2012},
 {'name': 'headr', 'founded_year': 2012},
 {'name': 'Pinger', 'founded_year': 2012},
 {'name': 'Widgetbox', 'founded_year': 2012},
 {'name': 'Mobiluck', 'founded_year': 2012},
 {'name': 'Skydeck', 'founded_year': 2012},
 {'name': 'Simplicant', 'founded_year': 2012},
 {'name': 'Springleap', 'founded_year': 2012},
 {'name': 'Jumbuck Entertainment', 'founded_year': 2012},
 {'nam

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [128]:
# Your Code

proj = {"_id": 0, "name": 1, "acquisition.price_amount": 1}
filtro = {"founded_day": {"$in": [1,2,3,4,5,6,7]}}
list(collection.find(filtro, proj).sort("acquisition.price_amount", -1).limit(10))


[{'name': 'Netscape', 'acquisition': {'price_amount': 4200000000}},
 {'name': 'PayPal', 'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos', 'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba', 'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini', 'acquisition': {'price_amount': 625000000}},
 {'name': 'Danger', 'acquisition': {'price_amount': 500000000}},
 {'name': 'Clearwell Systems', 'acquisition': {'price_amount': 410000000}},
 {'name': 'PrimeSense', 'acquisition': {'price_amount': 345000000}},
 {'name': 'Amobee', 'acquisition': {'price_amount': 321000000}},
 {'name': 'Vitrue', 'acquisition': {'price_amount': 300000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [130]:
# Your Code
proj = {"_id": 0, "name": 1, "number_of_employees": 1}
filtro = {"$and" : [{"category_code": "web"},{"number_of_employees": {"$gt": 4000}}]}
list(collection.find(filtro, proj).sort("number_of_employees", 1))

[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group', 'number_of_employees': 10000},
 {'name': 'Groupon', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'number_of_employees': 13600},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Experian', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [133]:
# Your Code
proj = {"_id": 0, "name":1, "acquisition.price_amount": 1}
filtro = {"$and" : [{"acquisition.price_amount":{"$gt":10000}}, {"acquisition.price_currency_code": "EUR"}]}
list(collection.find(filtro, proj))

[{'name': 'ZYB', 'acquisition': {'price_amount': 31500000}},
 {'name': 'Apertio', 'acquisition': {'price_amount': 140000000}},
 {'name': 'Greenfield Online', 'acquisition': {'price_amount': 40000000}},
 {'name': 'Mobile Trend', 'acquisition': {'price_amount': 7000000}},
 {'name': 'Citizenside', 'acquisition': {'price_amount': 30000}},
 {'name': 'Excite Europe', 'acquisition': {'price_amount': 2500000}},
 {'name': 'YelloYello', 'acquisition': {'price_amount': 2500000}},
 {'name': 'Webedia', 'acquisition': {'price_amount': 70000000}},
 {'name': 'Wayfinder', 'acquisition': {'price_amount': 24000000}},
 {'name': 'Branded Payment Solutions',
  'acquisition': {'price_amount': 3100000}},
 {'name': 'Tuenti Technologies', 'acquisition': {'price_amount': 70000000}},
 {'name': 'BioMed Central', 'acquisition': {'price_amount': 43400000}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [137]:
# Your Code
proj = {"_id": 0, "name": 1, "acquisition": 1}
filtro = {"$and": [{"acquisition.acquired_month": {"$in": [1,2,3]}}, {"acquisition.price_amount":{"$ne": None}}]}
list(collection.find(filtro, proj).limit(10))

[{'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   'acquired_day': None,
   'acquiring_company': {'name': 'Nielsen', 'permalink': 'nielsen'}}},
 {'name': 'iContact',
  'acquisition': {'price_amount': 169000000,
   'price_currency_code': 'USD',
   'term_code': 'cash_and_stock',
   'source_url': 'http://techcrunch.com/2012/02/28/vocus-buys-email-marketing-company-icontact-for-169-million/',
   'source_description': 'Vocus Buys Email Marketing Company iContact For $169 Million',
   'acquired_year': 2012,
   'acquired_month': 2,
   'a

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [138]:
# Your Code
proj = {"_id": 0, "name":1}
filtro = {"$and": [{"founded_year": {"$in" :[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]}}, {"acquisition.acquired_year": {"$gt": 2011}}]}
list(collection.find(filtro, proj))

[{'name': 'Wetpaint'},
 {'name': 'Digg'},
 {'name': 'Geni'},
 {'name': 'blogTV'},
 {'name': 'Revision3'},
 {'name': 'iContact'},
 {'name': 'Mashery'},
 {'name': 'Dailymotion'},
 {'name': 'KickApps'},
 {'name': 'Netvibes'},
 {'name': 'adBrite'},
 {'name': 'Meebo'},
 {'name': 'Seesmic'},
 {'name': 'Oodle'},
 {'name': 'Indeed'},
 {'name': 'Grockit'},
 {'name': 'Zenfolio'},
 {'name': 'Wishpot'},
 {'name': 'Retrevo'},
 {'name': 'SelectMinds'},
 {'name': 'TokBox'},
 {'name': 'Docstoc'},
 {'name': 'sezmi'},
 {'name': 'LocaModa'},
 {'name': 'OMGPOP'},
 {'name': 'Daylife'},
 {'name': 'Behance'},
 {'name': 'Attributor'},
 {'name': 'Viewdle'},
 {'name': 'Clickable'},
 {'name': 'Xobni'},
 {'name': 'Sermo'},
 {'name': 'Worklight'},
 {'name': 'Zimbra'},
 {'name': 'earthmine'},
 {'name': 'Whiskey Media'},
 {'name': 'CondoDomain'},
 {'name': 'Uptake'},
 {'name': 'AdReady'},
 {'name': 'The Climate Corporation'},
 {'name': 'Tumblr'},
 {'name': 'JackBe'},
 {'name': 'Rollbase'},
 {'name': 'Marakana'},
 {'

### 20. All the companies that have been 'deadpooled' after the third year.

In [157]:
proj = {"_id": 0, "name": 1, "founded_year": 1, "deadpooled_year": 1}
filtro = {"$and": [{"founded_year":{"$ne": None}}, {"deadpooled_year":{"$ne": None}}]}
dp = list(collection.find(filtro, proj))

In [148]:
import pandas as pd

In [158]:
df = pd.DataFrame(dp)

In [159]:
df

,name,founded_year,deadpooled_year
0,Wetpaint,2005,1
1,AdventNet,1996,2
2,Zoho,2005,3
3,Omnidrive,2005,2008
4,Babelgum,2007,2013
...,...,...,...
706,Bookrags,1999,1991
707,tribalX,2009,2010
708,Prolify,2002,2007
709,Advanced Power Projects,2009,2011


In [164]:
df.loc[df["deadpooled_year"]-df["founded_year"]>=3, ["name"]]

,name
3,Omnidrive
4,Babelgum
6,Thoof
7,Wesabe
8,Jangl SMS
...,...
700,SpeakSoft
702,Tagito
703,Nordic Windpower
705,Nethra Imaging
